In [12]:
# from git import Repo
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import Language
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [13]:
import pandas as pd

df = pd.read_csv("/Users/borisyu/Desktop/github/AutoFinViz/example/data/Stock_price_TSLA.csv")

In [14]:
loader = GenericLoader.from_filesystem(
    "/Users/borisyu/Desktop/github/AutoFinViz/notebooks/graph",
    glob="**/*",
    suffixes=[".py"],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=500),
)
documents = loader.load()
len(documents)

5

In [15]:
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=2000, chunk_overlap=200
)
texts = python_splitter.split_documents(documents)
len(texts)

5

In [16]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

db = Chroma.from_documents(texts, OpenAIEmbeddings(disallowed_special=()))
retriever = db.as_retriever(
    search_type="mmr",  # Also test "similarity"
    search_kwargs={"k": 5},
)

In [17]:
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory

llm = ChatOpenAI(model_name="gpt-3.5-turbo")


In [46]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# RAG template
prompt_RAG = """
    You are a proficient python developer. Respond with the syntactically correct code for to the question below. Make sure you follow these rules:
    1. Use context to understand the APIs and how to use it & apply.
    2. Do not add license information to the output code.
    3. Do not include colab code in the output.
    4. Ensure all the requirements in the question are met.

    Question:
    {question}

    Context:
    {context}

    Helpful Response :
    """

prompt_RAG_tempate = PromptTemplate(
    template=prompt_RAG, input_variables=["context", "question"]
)

qa_chain = RetrievalQA.from_llm(
    llm=llm, prompt=prompt_RAG_tempate, retriever=retriever, return_source_documents=True
)


graph = "Multi-line plot"

title = "Sample Title"

question = f"""Write a python function that takes df dataframe and plot {graph} in plotly library.
USE the X_column {X_columns}, Y_columns {Y_columns}
SET title as "{title}".
ONLY Return executable PYTHON code.
NAME the function as plot(df)

TEMPLATE:

import ...

def plot(df: pd.dataframe):

    return fig

fig = plot(df)
fig.write_image("../example/figures/{title}.png)
fig.show()
"""

results = qa_chain({"query": question})
code = results["result"]

try:
    fig = exec(code)
except Exception as e:
    print(f"An error occurred: {e}")

Number of requested results 20 is greater than number of elements in index 10, updating n_results = 10


In [49]:
print(results["query"])

Write a python function that takes df dataframe and plot Multi-line plot in plotly library.
USE the X_column ['Date'], Y_columns ['Open', 'Close', 'High', 'Low']
SET title as "Sample Title".
ONLY Return executable PYTHON code.
NAME the function as plot(df)

TEMPLATE:

import ...

def plot(df: pd.dataframe):

    return fig

fig = plot(df)
fig.write_image("../example/figures/Sample Title.png)
fig.show()



In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# RAG template
prompt_RAG = """
    You are a proficient python developer. Respond with the syntactically correct code for to the question below. Make sure you follow these rules:
    1. Use context to understand the APIs and how to use it & apply.
    2. Do not add license information to the output code.
    3. Do not include colab code in the output.
    4. Ensure all the requirements in the question are met.

    Question:
    {question}

    Context:
    {context}

    Helpful Response :
    """

prompt_RAG_tempate = PromptTemplate(
    template=prompt_RAG, input_variables=["context", "question"]
)

qa_chain = RetrievalQA.from_llm(
    llm=llm, prompt=prompt_RAG_tempate, retriever=retriever, return_source_documents=True
)



graph = "Multi-line plot"

title = "Sample Title"

question = f"""Write a python function that takes df dataframe and plot {graph} in plotly library.
USE the X_column {X_columns}, Y_columns {Y_columns}
SET title as "{title}".
ONLY Return executable PYTHON code.
NAME the function as plot(df)

TEMPLATE:

import ...

def plot(df: pd.dataframe):

    return fig

fig = plot(df)
fig.write_image("../example/figures/{title}.png)
fig.show()
"""

results = qa_chain({"query": question})
code = results["result"]

try:
    fig = exec(code)
except Exception as e:
    print(f"An error occurred: {e}")

## NEW Question Formulator 

In [39]:
summary = summary = [{'column': 'Date',
  'properties': {'dtype': 'date',
   'min': '2022-11-21',
   'max': '2023-11-17',
   'num_unique_values': 250,
   'samples': ['2022-11-21', '2022-11-22', '2022-11-23'],
   'semantic_type': '',
   'description': ''}},
 {'column': 'Open',
  'properties': {'dtype': 'number',
   'min': 103.0,
   'max': 296.040009,
   'num_unique_values': 248,
   'samples': [175.850006, 168.630005, 173.570007],
   'semantic_type': '',
   'description': ''}},
 {'column': 'High',
  'properties': {'dtype': 'number',
   'min': 111.75,
   'max': 299.290009,
   'num_unique_values': 246,
   'samples': [176.770004, 170.919998, 183.619995],
   'semantic_type': '',
   'description': ''}},
 {'column': 'Low',
  'properties': {'dtype': 'number',
   'min': 101.809998,
   'max': 289.519989,
   'num_unique_values': 247,
   'samples': [167.539993, 166.190002, 172.5],
   'semantic_type': '',
   'description': ''}},
 {'column': 'Close',
  'properties': {'dtype': 'number',
   'min': 108.099998,
   'max': 293.339996,
   'num_unique_values': 247,
   'samples': [167.869995, 169.910004, 183.199997],
   'semantic_type': '',
   'description': ''}},
 {'column': 'Adj Close',
  'properties': {'dtype': 'number',
   'min': 108.099998,
   'max': 293.339996,
   'num_unique_values': 247,
   'samples': [167.869995, 169.910004, 183.199997],
   'semantic_type': '',
   'description': ''}},
 {'column': 'Volume',
  'properties': {'dtype': 'number',
   'min': 50672700,
   'max': 306590600,
   'num_unique_values': 250,
   'samples': [92882700, 78452300, 109536700],
   'semantic_type': '',
   'description': ''}},
 {'column': 'Daily Range',
  'properties': {'dtype': 'number',
   'min': 2.910004000000015,
   'max': 20.38000500000001,
   'num_unique_values': 240,
   'samples': [9.23001099999999, 4.729996, 11.119994999999989],
   'semantic_type': '',
   'description': ''}},
 {'column': 'Percentage Change',
  'properties': {'dtype': 'number',
   'min': -0.08753273328663176,
   'max': 0.09766988349514556,
   'num_unique_values': 250,
   'samples': [-0.045379645878431296,
    0.007590576777839597,
    0.055481878271745375],
   'semantic_type': '',
   'description': ''}},
 {'column': 'Volume Weighted Average Price',
  'properties': {'dtype': 'number',
   'min': 138.933071859511,
   'max': 203.68287803146123,
   'num_unique_values': 250,
   'samples': [167.869995, 168.80409134966993, 174.41832731024735],
   'semantic_type': '',
   'description': ''}}]

n = 4

list_of_graph = ["Multi-lineplot", "OHLC Chart", "Simple Candlestick", "Time Series data with Histograms", "Time Series with Rangeslider"]

In [40]:
format_instruction = """
    THE OUTPUT MUST BE A CODE SNIPPET OF A VALID LIST OF JSON OBJECTS. IT MUST USE THE FOLLOWING FORMAT:

    ```
    [
        { "index": 0,  "title": "How", "visualization_type": "histogram", "x_axis": ["col_1", "col_2"], "y_axis": ["col_3", "col_4"]},
    ]
    ```
    THE OUTPUT SHOULD ONLY USE THE JSON FORMAT ABOVE.
"""

user_prompt = f"""
    
    Given the summary of dataframe: {summary}

    Select a plot among {list_of_graph} as the visulization_type
    SET a question as the title of the plot.
    CHOOSE columns name for x axis and y axis.



    FOLLOW this output format instruction:{format_instruction}


    'visualization_type' can ONLY within {list_of_graph}.

"""

In [41]:
## Get API from 'apikey' file
filename = '../apikey'
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)
api_key = get_file_contents(filename)
# print("Your API key is: %s" % (api_key))

from openai import OpenAI

client = OpenAI(
  api_key= api_key
)

In [42]:
SYSTEM_INSTRUCTIONS = """
You are an experienced financial data analyst who can generate a given number of insightful PLOTs about data, when given a summary of the data. 
MAKE SURE THE QUESTION IS A FINANCIAL/Economic QUESTION INSTEAD OF A STATISTIC QUESTION.
"""

In [43]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages = [
    {"role": "system", "content": SYSTEM_INSTRUCTIONS},
    {"role": "assistant",
        "content": f"{user_prompt}"
    }
    ]
  )

print(completion.choices[0].message.content)

[
    { "index": 0,  "title": "Stock Price Trend Over Time", "visualization_type": "Multi-lineplot", "x_axis": ["Date"], "y_axis": ["Close", "Volume"]},
    { "index": 1,  "title": "Stock Price Range - OHLC", "visualization_type": "OHLC Chart", "x_axis": ["Date"], "y_axis": ["Open", "High", "Low", "Close"]},
    { "index": 2,  "title": "Stock Price Movement - Candlestick", "visualization_type": "Simple Candlestick", "x_axis": ["Date"], "y_axis": ["Open", "High", "Low", "Close"]},
    { "index": 3,  "title": "Stock Price Distribution Over Time", "visualization_type": "Time Series data with Histograms", "x_axis": ["Date"], "y_axis": ["Close"]}
]


In [24]:
import json
from langchain.prompts import ChatPromptTemplate

model = ChatOpenAI(model_name="gpt-3.5-turbo")
prompt_template = ChatPromptTemplate.from_template(user_prompt)

chain = prompt_template | model


In [26]:
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {' "index"': {'title': ' "Index"', 'type': 'string'},
  "'column'": {'title': "'Column'", 'type': 'string'}}}

In [28]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser


In [29]:
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}